In [1]:
import numpy as np
import cv2
import pickle 
from cv2 import VideoCapture
from cv2 import waitKey
from keras.models import load_model

In [2]:
framewidth = 640
frameHeight = 480
brightness = 180
threshold = 0.90
dim = (framewidth,frameHeight)
font = cv2.FONT_HERSHEY_SIMPLEX

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3, framewidth)
cap.set(4, frameHeight)
cap.set(10, brightness)
model = load_model("C:/Users/k2h2j3/Model/Traffic_Model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 60)        1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 60)        90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 60)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 30)        16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 30)          8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 30)         0         
 2D)                                                    

In [4]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

def getCalssName(classIndex):
    if classIndex == 0: 
        return 'speed Limit 20km/h'
    elif classIndex == 1: 
        return 'speed Limit 30km/h'
    elif classIndex == 2: 
        return 'speed Limit 50km/h'
    elif classIndex == 3: 
        return 'speed Limit 60km/h'
    elif classIndex == 4: 
        return 'speed Limit 70km/h'
    elif classIndex == 5: 
        return 'speed Limit 80km/h'
    elif classIndex == 6: 
        return 'end of speed Limit 80km/h'
    elif classIndex == 7: 
        return 'speed Limit 100km/h'
    elif classIndex == 8: 
        return 'speed Limit 120km/h'
    elif classIndex == 9: 
        return 'No passing'
    elif classIndex == 10:
        return 'No passing for vehicles over 3.5 metric tons'
    elif classIndex == 11: 
        return 'Right-of-way at the next intersection'
    elif classIndex == 12: 
        return 'prioirty road'
    elif classIndex == 13: 
        return 'Yield'
    elif classIndex == 14: 
        return 'Stop'
    elif classIndex == 15: 
        return 'No vehicles'
    elif classIndex == 16:
        return 'Veh > 3.5 tons prohibited'
    elif classIndex == 17:
        return 'No entry'
    elif classIndex == 18:
        return 'General caution'
    elif classIndex == 19:
        return 'Dangerous curve left'
    elif classIndex == 20: 
        return 'Dangerous curve right'
    elif classIndex == 21:
        return 'Double curve'
    elif classIndex == 22:
        return 'Bumpy road'
    elif classIndex == 23:
        return 'Slippery road'
    elif classIndex == 24:
        return 'Road narrows on the right'
    elif classIndex == 25: 
        return 'Road work'
    elif classIndex == 26:
        return 'Traffic signals'
    elif classIndex == 27: 
        return 'Pedetrians'
    elif classIndex == 28:
        return 'Children crossing'
    elif classIndex == 29:
        return 'Bicycles crossing'
    elif classIndex == 30:
        return 'Beware of ice/snow'
    elif classIndex == 31:
        return 'Wild animals crossing'
    elif classIndex == 32:
        return 'End speed + passing limits'
    elif classIndex == 33:
        return 'Turn right ahead'
    elif classIndex == 34:
        return 'Turn left ahead'
    elif classIndex == 35:
        return 'Ahead only'
    elif classIndex == 36:
        return 'Go straight or right'
    elif classIndex == 37:
        return 'Go straight or left'
    elif classIndex == 38:
        return 'keep right'
    elif classIndex == 39:
        return 'keep left'
    elif classIndex == 40:
        return 'Roundabout mandatory'
    elif classIndex == 41:
        return 'End of no passing'
    elif classIndex == 42:
        return 'End no passing veh > 3.5 tons'

In [5]:
while True:
 
# READ IMAGE
    success,resized = cap.read()

    img = np.asarray(resized)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)
    cv2.imshow("Processed Image", img)
    img = img.reshape(1, 32, 32, 1)
    cv2.putText(resized, "CLASS: " , (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(resized, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    predictions = model.predict(img)
    classIndex = np.argmax(predictions[0])
    probabilityValue =np.amax(predictions)
   
    #if probabilityValue > threshold:
    #    cv2.putText(imgOrignal,str(classIndex)+" "+str(getCalssName(classIndex)), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA) 
    #    cv2.putText(imgOrignal, str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)        
    #cv2.imshow("Result", imgOrignal)
    gray = cv2.medianBlur(cv2.cvtColor(cap.read()[1], cv2.COLOR_BGR2GRAY),5)
    resized = cv2.resize(gray,dim,interpolation = cv2.INTER_AREA)
    edges = cv2.Canny(gray,100,200)
    circ = cv2.HoughCircles(resized,cv2.HOUGH_GRADIENT,1,50,param1=70,param2=100,
                              minRadius=0,maxRadius=0)
    if probabilityValue > threshold:
        cv2.putText(resized,str(classIndex)+" "+str(getCalssName(classIndex)), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA) 
        cv2.putText(resized,str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
 
    cv2.imshow('video',resized)
    if circ is not None:
        circ = np.uint16(np.around(circ))[0,:]
        print(circ)
        for j in circ:
            cv2.circle(resized, (j[0], j[1]), j[2], (0, 255, 0), 2)
        cv2.imshow('video',resized)
    


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

[[524 186  76]]
[[514 126  75]]
[[524 118  93]]
[[518 116  94]]
[[512 124  79]]
[[510 140  81]]
[[506 158  95]]
[[510 176  82]]
[[516 186  98]]
[[514 192  81]]
[[514 196  80]]
[[508 196  98]]
[[500 200  98]]
[[484 204  99]]
[[464 200 101]]
[[442 200  95]]
[[418 184 102]]
[[400 176 102]]
[[388 168 104]]
[[382 164  82]]
[[376 150 103]]
[[370 144 102]]
[[358 142  99]]
[[346 136  87]]
[[322 122  92]]
[[288 122  89]]
[[266 120 111]]
[[266 130 111]]
[[236 142 102]]
[[182 188 105]]
[[150 216 132]]
[[152 216 117]]
[[154 226 118]]
[[162 224 116]]
[[180 234 105]]
[[202 228 107]]
[[232 222 110]]
[[264 224 109]]
[[290 222 113]]
[[306 230 143]]
[[324 228 142]]
[[324 226 146]]
[[314 214 151]]
[[290 198 122]]
[[276 190 124]]
[[262 178 118]]
[[266 178 146]]
[[270 214 146]]
[[278 220 148]]
[[286 222 121]]
[[304 226 152]]
[[316 228 123]]
[[320 204 122]]
[[318 200 117]]
[[310 204 120]]
[[310 204 157]]
[[304 206 156]]
[[300 216 125]]
[[290 224 127]]
[[288 226 160]]
[[276 244 129]]
[[276 240 162]]
[[280 23